#### Note

In [ ]:
# pd.set_option('display.max_rows', 100) # 최대 display option set
# pd.reset_option('display.max_rows') # 최대 display option reset

# from datetime import timedelta
# datetime.datetime(2019,12,31) - datetime.datetime(2008,1,1)

#### Import

In [9]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### ABSI 정의
체형지수(ABSI) = 허리둘레 / (BMI × 2/3) × (신장×1/2)

#### 07 - 20 기본 DB concat              
기타 검진변수들 일치하는 시점부터 적당히 잘라서 합치자        
- 07 이전에는 wc가 없음

필수변수   
- HE_ht         
- HE_wt        
- HE_BMI         
- HE_wc

건보공단이랑 일치하는 변수 list

- year : 조사연도
- ID : 개인 아이디
- psu : 조사구번호
- HE_ht : 신장
- HE_wt : 체중        


- HE_BMI : 체질량지수
- HE_sbp_tr : 최종 수축기 혈압(보정값)
- HE_dbp_tr : 최종 이완기 혈압(보정값)
- HE_obe : 비만유병여부(19세이상) - categorical
- HE_Upro : 요단백  - categorical
- HE_HbA1c : 당화혈색소
- HE_glu : 공복혈당
- HE_chol : 총콜레스테롤
- HE_ast : GOT
- HE_alt : GPT
- HE_tb : 흉부영상 판독결과  - categorical
- HE_HP_tr : 고혈압 유병여부(30세이상, 보정값) - categorical
- DI3_lt : 뇌졸중 평생유병여부 - categorical
- DI4_lt : 심근경색증 또는 협심증 평생유병여부 - categorical
- HE_DM : 당뇨병유병여부(19세이상) - categorical
- HE_wc : 허리둘레
- HE_HDL_st2 : 전환식 HDL-콜레스테롤
- HE_LDL_drct : LDL콜레스테롤_직접검사
- HE_crea : 혈중크레아티닌
- D_1_1 : 주관적 건강상태 - categorical       


- L_BR1 : 조사 1일전 아침식사 섭취여부 - categorical
- L_LN1 : 조사 1일전 점심식사 섭취여부 - categorical
- L_DN1 : 조사 1일전 저녁식사 섭취여부 - categorical
--> 이거 세개를 하나로 축소해야 함 (건보공단 변수는 하나임)


- N_NA : 1일 나트륨 섭취량(mg)     


- F_BEEF : 쇠고기 (국, 탕, 찌개, 편육, 장조림, 구이,볶음, 비프까스, 튀김, 찜 포함) - categorical
- F_CHICK : 닭고기 (삼계탕, 백숙, 찜, 튀김, 조림, 볶음 포함) - categorical
- F_PORK : 돼지고기 (찌개, 구이, 볶음, 돈까스, 튀김 포함) - categorical
- --> 이거 세개를 하나로 축소해야 함 (건보공단 변수는 하나임)     


- BS3_1 : (성인) 현재흡연 여부 - categorical
- BS3_2 : (성인) 현재흡연자 하루 평균 흡연량 - numerical 이나, 888 / 999 로 처리된 거 0으로 전처리 해줘야 함    


- BD1_11 : (만12세이상) 1년간 음주빈도 - categorical
- BD2_1 : (만12세이상) 한번에 마시는 음주량 - categorical     


- BE3_21 : 1주일간 중등도 신체활동 일수 - categorical
- BE3_22 : 중등도 신체활동 지속시간(시간) - numerical 이나, 88 / 99 로 처리된 거 0으로 전처리 해줘야 함     


- BP1 : 평소 스트레스 인지 정도 - categorical    


- DI3_dg : 뇌졸중 의사진단여부 - categorical
- DI4_dg : 심근경색증 또는 협심증 의사진단여부 - categorical
- DI1_dg : 고혈압 의사진단여부 - categorical
- DE1_dg : 당뇨병 의사진단여부 (소아청소년 포함) - categorical
- DI2_dg : 고지혈증 의사진단여부 - categorical   


- DI3_pt : 뇌졸중 치료 - categorical
- DI4_pt : 심근경색증 또는 협심증 치료 - categorical
- DI1_pt : 고혈압 치료 - categorical
- DE1_pt : 당뇨병 치료 - categorical  
- DI2_pt : 고지혈증  치료 - categorical      


- HE_hepaB : B형간염표면항원 양성여부(10세이상) - categorical
- BS6_2_2 : (성인) 과거흡연자 흡연 기간(월) - numerical 이나, 88 / 99 로 처리된 거 0으로 전처리 해줘야 함
- BS6_3 : (성인) 과거흡연자 하루 평균 흡연량 - numerical 이나, 88 / 99 로 처리된 거 0으로 전처리 해줘야 함
- BS2_1 : (성인) 흡연 시작연령 - numerical 이나, 88 / 99 로 처리된 거 0으로 전처리 해줘야 함     


- pa_high : 격렬한 신체활동 실천율 - categorical
- pa_mid : 중등도 신체활동 실천율 - categorical  
- pa_walk : 걷기 실천율 - categorical    


- DJ1_dg : 폐결핵 또는 폐외결핵 의사진단여부 (소아, 청소년 포함) - categorical
- DJ1_pt : 폐결핵 또는 폐외결핵 치료 - categorical

In [14]:
basic1 = pd.read_csv('D:\\국민건강영양조사\\국민건강영양조사_기수별통합\\1998-2019 기본DB\\기본DB_1기 (1998).csv', encoding = 'euc-kr', low_memory = False)
basic2 = pd.read_csv('D:\\국민건강영양조사\\국민건강영양조사_기수별통합\\1998-2019 기본DB\\기본DB_2기 (2001).csv', encoding = 'euc-kr', low_memory = False)
basic3 = pd.read_csv('D:\\국민건강영양조사\\국민건강영양조사_기수별통합\\1998-2019 기본DB\\기본DB_3기 (2005).csv', encoding = 'euc-kr', low_memory = False)
basic4 = pd.read_csv('D:\\국민건강영양조사\\국민건강영양조사_기수별통합\\1998-2019 기본DB\\기본DB_4기 (2007-2009).csv', encoding = 'euc-kr', low_memory = False)
basic5 = pd.read_csv('D:\\국민건강영양조사\\국민건강영양조사_기수별통합\\1998-2019 기본DB\\기본DB_5기 (2010-2012).csv', encoding = 'euc-kr', low_memory = False)
basic6 = pd.read_csv('D:\\국민건강영양조사\\국민건강영양조사_기수별통합\\1998-2019 기본DB\\기본DB_6기 (2013-2015).csv', encoding = 'euc-kr', low_memory = False)
basic7 = pd.read_csv('D:\\국민건강영양조사\\국민건강영양조사_기수별통합\\1998-2019 기본DB\\기본DB_7기 (2016-2017).csv', encoding = 'euc-kr', low_memory = False)
basic18 = pd.read_sas('D:\\국민건강영양조사\\1998-2020 기본DB\\hn18_all.sas7bdat')
basic19 = pd.read_sas('D:\\국민건강영양조사\\1998-2020 기본DB\\hn19_all.sas7bdat')
basic20 = pd.read_sas('D:\\국민건강영양조사\\1998-2020 기본DB\\hn20_all.sas7bdat')

In [64]:
# 4기부터 wc 있어서 나중에 추가

basic4_p = basic4[['year', 'ID', 'sex', 'age', 'psu', 'HE_ht', 'HE_wt', 'HE_BMI', 'HE_obe', 'HE_Upro',
                      'HE_HbA1c', 'HE_glu', 'HE_chol', 'HE_ast', 'HE_alt', 
                      'HE_wc', 'HE_HDL_st2', 'HE_LDL_drct', 'HE_crea', 'D_1_1', 
                      'N_NA', 'BS3_1', 'BS3_2', 'BD1_11', 'BD2_1', 
                      'BP1', 'DI3_dg', 'DI4_dg', 'DI1_dg', 'DE1_dg', 'DI2_dg', 'DI3_pt', 'DI4_pt', 'DI1_pt', 'DE1_pt',
                      'DI2_pt', 'HE_hepaB', 'BS6_2_2', 'BS6_3', 'BS2_1']]

In [29]:
# 'DJ1_dg', 'DJ1_pt', 'HW_MS' 삭제 (없음) 
# 'HE_tb', 'F_BEEF', 'HE_dbp_tr', 'L_LN1', 'L_DN1', 'DI4_lt', 'F_CHICK', 'F_PORK', 'HE_sbp_tr', 'L_BR1', 'DI3_lt', 'HE_HP_tr' 삭제 (6기에 없음)
# 'pa_mid', 'BE3_21', 'pa_high', 'BE3_22', 'pa_walk' 삭제 (7기에 없음)
# 'HE_DM' 삭제 (19년도에 없음)
basic5_p = basic5[['year', 'ID', 'sex', 'age', 'psu', 'HE_ht', 'HE_wt', 'HE_BMI', 'HE_obe', 'HE_Upro',
                      'HE_HbA1c', 'HE_glu', 'HE_chol', 'HE_ast', 'HE_alt', 
                      'HE_wc', 'HE_HDL_st2', 'HE_LDL_drct', 'HE_crea', 'D_1_1', 
                      'N_NA', 'BS3_1', 'BS3_2', 'BD1_11', 'BD2_1', 
                      'BP1', 'DI3_dg', 'DI4_dg', 'DI1_dg', 'DE1_dg', 'DI2_dg', 'DI3_pt', 'DI4_pt', 'DI1_pt', 'DE1_pt',
                      'DI2_pt', 'HE_hepaB', 'BS6_2_2', 'BS6_3', 'BS2_1']]

In [30]:
# 'HE_tb', 'F_BEEF', 'HE_dbp_tr', 'L_LN1', 'L_DN1', 'DI4_lt', 'F_CHICK', 'F_PORK', 'HE_sbp_tr', 'L_BR1', 'DI3_lt', 'HE_HP_tr' 삭제 (없음)
# 'pa_mid', 'BE3_21', 'pa_high', 'BE3_22', 'pa_walk' 삭제 (7기에 없음)
# 'HE_DM' 삭제 (19년도에 없음)
basic6_p = basic6[['year', 'ID', 'sex', 'age', 'psu', 'HE_ht', 'HE_wt', 'HE_BMI', 'HE_obe', 'HE_Upro',
                      'HE_HbA1c', 'HE_glu', 'HE_chol', 'HE_ast', 'HE_alt', 
                      'HE_wc', 'HE_HDL_st2', 'HE_LDL_drct', 'HE_crea', 'D_1_1', 
                      'N_NA', 'BS3_1', 'BS3_2', 'BD1_11', 'BD2_1', 
                      'BP1', 'DI3_dg', 'DI4_dg', 'DI1_dg', 'DE1_dg', 'DI2_dg', 'DI3_pt', 'DI4_pt', 'DI1_pt', 'DE1_pt',
                      'DI2_pt', 'HE_hepaB', 'BS6_2_2', 'BS6_3', 'BS2_1']]

In [31]:
# 'pa_mid', 'BE3_21', 'pa_high', 'BE3_22', 'pa_walk' 삭제 (없음)
# 'HE_DM' 삭제 (19년도에 없음)
basic7_p = basic7[['year', 'ID', 'sex', 'age', 'psu', 'HE_ht', 'HE_wt', 'HE_BMI', 'HE_obe', 'HE_Upro',
                      'HE_HbA1c', 'HE_glu', 'HE_chol', 'HE_ast', 'HE_alt', 
                      'HE_wc', 'HE_HDL_st2', 'HE_LDL_drct', 'HE_crea', 'D_1_1', 
                      'N_NA', 'BS3_1', 'BS3_2', 'BD1_11', 'BD2_1', 
                      'BP1', 'DI3_dg', 'DI4_dg', 'DI1_dg', 'DE1_dg', 'DI2_dg', 'DI3_pt', 'DI4_pt', 'DI1_pt', 'DE1_pt',
                      'DI2_pt', 'HE_hepaB', 'BS6_2_2', 'BS6_3', 'BS2_1']]

In [32]:
# 'HE_DM' 삭제 (19년도에 없음)
basic18_p = basic18[['year', 'ID', 'sex', 'age', 'psu', 'HE_ht', 'HE_wt', 'HE_BMI', 'HE_obe', 'HE_Upro',
                      'HE_HbA1c', 'HE_glu', 'HE_chol', 'HE_ast', 'HE_alt', 
                      'HE_wc', 'HE_HDL_st2', 'HE_LDL_drct', 'HE_crea', 'D_1_1', 
                      'N_NA', 'BS3_1', 'BS3_2', 'BD1_11', 'BD2_1', 
                      'BP1', 'DI3_dg', 'DI4_dg', 'DI1_dg', 'DE1_dg', 'DI2_dg', 'DI3_pt', 'DI4_pt', 'DI1_pt', 'DE1_pt',
                      'DI2_pt', 'HE_hepaB', 'BS6_2_2', 'BS6_3', 'BS2_1']]

In [33]:
# 'HE_DM' 삭제 
basic19_p = basic19[['year', 'ID', 'sex', 'age', 'psu', 'HE_ht', 'HE_wt', 'HE_BMI', 'HE_obe', 'HE_Upro',
                      'HE_HbA1c', 'HE_glu', 'HE_chol', 'HE_ast', 'HE_alt', 
                      'HE_wc', 'HE_HDL_st2', 'HE_LDL_drct', 'HE_crea', 'D_1_1', 
                      'N_NA', 'BS3_1', 'BS3_2', 'BD1_11', 'BD2_1', 
                      'BP1', 'DI3_dg', 'DI4_dg', 'DI1_dg', 'DE1_dg', 'DI2_dg', 'DI3_pt', 'DI4_pt', 'DI1_pt', 'DE1_pt',
                      'DI2_pt', 'HE_hepaB', 'BS6_2_2', 'BS6_3', 'BS2_1']]

In [34]:
basic20_p = basic20[['year', 'ID', 'sex', 'age', 'psu', 'HE_ht', 'HE_wt', 'HE_BMI', 'HE_obe', 'HE_Upro',
                      'HE_HbA1c', 'HE_glu', 'HE_chol', 'HE_ast', 'HE_alt', 
                      'HE_wc', 'HE_HDL_st2', 'HE_LDL_drct', 'HE_crea', 'D_1_1', 
                      'N_NA', 'BS3_1', 'BS3_2', 'BD1_11', 'BD2_1', 
                      'BP1', 'DI3_dg', 'DI4_dg', 'DI1_dg', 'DE1_dg', 'DI2_dg', 'DI3_pt', 'DI4_pt', 'DI1_pt', 'DE1_pt',
                      'DI2_pt', 'HE_hepaB', 'BS6_2_2', 'BS6_3', 'BS2_1']]

In [81]:
basic0719 = pd.concat([basic4_p, basic5_p, basic6_p, basic7_p, basic18_p, basic19_p], axis = 0)
basic0719

,year,ID,sex,age,psu,HE_ht,HE_wt,HE_BMI,HE_obe,HE_Upro,...,DI2_dg,DI3_pt,DI4_pt,DI1_pt,DE1_pt,DI2_pt,HE_hepaB,BS6_2_2,BS6_3,BS2_1
0,2007.0,b'A548480901',1.0,55.0,b'A548',159.9,60.6,23.701493,2.0,0.0,...,8.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,88.0
1,2007.0,b'A548480902',2.0,50.0,b'A548',157.0,51.4,20.852773,2.0,0.0,...,8.0,8.0,8.0,0.0,8.0,8.0,0.0,99.0,1.0,22.0
2,2007.0,b'A548480903',1.0,23.0,b'A548',NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007.0,b'A548480904',2.0,21.0,b'A548',161.1,50.0,19.265440,2.0,1.0,...,8.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,14.0
4,2007.0,b'A548560901',2.0,35.0,b'A548',164.0,73.3,27.253123,3.0,NaN,...,8.0,8.0,8.0,8.0,8.0,8.0,0.0,3.0,3.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8105,2019.0,b'R504381902',2.0,48.0,b'R504',155.1,49.4,20.535409,2.0,1.0,...,0.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,888.0
8106,2019.0,b'R504393801',1.0,47.0,b'R504',166.5,62.1,22.400779,2.0,0.0,...,0.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,888.0
8107,2019.0,b'R504393802',2.0,43.0,b'R504',162.5,40.6,15.375148,1.0,0.0,...,0.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,888.0
8108,2019.0,b'R504393803',2.0,16.0,b'R504',162.5,39.2,14.844970,NaN,0.0,...,8.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,888.0


In [65]:
basic0720 = pd.concat([basic4_p, basic5_p, basic6_p, basic7_p, basic18_p, basic19_p, basic20_p], axis = 0)
basic0720

,year,ID,sex,age,psu,HE_ht,HE_wt,HE_BMI,HE_obe,HE_Upro,...,DI2_dg,DI3_pt,DI4_pt,DI1_pt,DE1_pt,DI2_pt,HE_hepaB,BS6_2_2,BS6_3,BS2_1
0,2007.0,b'A548480901',1.0,55.0,b'A548',159.9,60.6,23.701493,2.0,0.0,...,8.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,88.0
1,2007.0,b'A548480902',2.0,50.0,b'A548',157.0,51.4,20.852773,2.0,0.0,...,8.0,8.0,8.0,0.0,8.0,8.0,0.0,99.0,1.0,22.0
2,2007.0,b'A548480903',1.0,23.0,b'A548',NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007.0,b'A548480904',2.0,21.0,b'A548',161.1,50.0,19.265440,2.0,1.0,...,8.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,14.0
4,2007.0,b'A548560901',2.0,35.0,b'A548',164.0,73.3,27.253123,3.0,NaN,...,8.0,8.0,8.0,8.0,8.0,8.0,0.0,3.0,3.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7354,2020.0,b'R804366601',2.0,61.0,b'R804',155.2,57.6,23.913275,3.0,0.0,...,0.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,888.0
7355,2020.0,b'R804368301',2.0,51.0,b'R804',144.7,51.8,24.739601,3.0,0.0,...,0.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,888.0
7356,2020.0,b'R804368302',2.0,24.0,b'R804',148.6,57.7,26.129927,4.0,0.0,...,0.0,8.0,8.0,8.0,8.0,8.0,0.0,88.0,888.0,14.0
7357,2020.0,b'R804371701',1.0,65.0,b'R804',171.5,76.3,25.941572,4.0,0.0,...,1.0,8.0,8.0,8.0,8.0,1.0,0.0,0.0,20.0,18.0


In [66]:
basic0720['HE_wc'].describe()

count    106265.000000
mean         77.497386
std          13.809327
min          19.600000
25%          69.400000
50%          78.900000
75%          87.100000
max         166.100000
Name: HE_wc, dtype: float64

In [82]:
# ABSI 생성
# 체형지수(ABSI) = 허리둘레 / (BMI × 2/3) × (신장×1/2)

basic0719['ABSI'] = basic0719['HE_wc'] / basic0719['HE_BMI'] * (2/3) * basic0719['HE_ht'] * (1/2)
basic0720['ABSI'] = basic0720['HE_wc'] / basic0720['HE_BMI'] * (2/3) * basic0720['HE_ht'] * (1/2)

In [68]:
basic0720['ABSI'].describe()

count    104991.000000
mean        180.514010
std          28.758365
min          45.336920
25%         167.610315
50%         183.493556
75%         198.802251
max         370.244931
Name: ABSI, dtype: float64

In [85]:
basic0719['ABSI'].describe()

count    98307.000000
mean       179.917198
std         29.085291
min         45.336920
25%        167.087031
50%        183.044992
75%        198.443261
max        370.244931
Name: ABSI, dtype: float64

In [86]:
# 07-19 조사인원 105,732명

len(basic0719['ID'].unique())

105732

In [69]:
# 07-20 조사인원 113,091명

len(basic0720['ID'].unique())

113091

In [88]:
# 남자 48,137명
# 여자 57,595명

basic0719['sex'].value_counts()

2.0    57595
1.0    48137
Name: sex, dtype: int64

In [70]:
# 남자 51,551명
# 여자 61,540명

basic0720['sex'].value_counts()

2.0    61540
1.0    51551
Name: sex, dtype: int64

In [89]:
# 연령대 1 ~ 80
# mean 41, std 22

basic0719['age'].describe()

count    105732.000000
mean         41.244590
std          22.774104
min           1.000000
25%          22.000000
50%          42.000000
75%          60.000000
max          80.000000
Name: age, dtype: float64

In [71]:
# 연령대 1 ~ 80
# mean 41, std 22

basic0720['age'].describe()

count    113091.000000
mean         41.497113
std          22.781171
min           1.000000
25%          22.000000
50%          43.000000
75%          60.000000
max          80.000000
Name: age, dtype: float64

#### 성별, 연령대별 ABSI mean, std 비교

In [90]:
# 20세 미만 남성

basic0719.loc[(basic0719['sex'] == 1) & (basic0719['age'] < 20), 'ABSI'].describe()

count    11361.000000
mean       156.872321
std         41.128152
min         45.336920
25%        127.587773
50%        162.148550
75%        189.765064
max        266.730539
Name: ABSI, dtype: float64

In [73]:
# 20세 미만 남성

basic0720.loc[(basic0720['sex'] == 1) & (basic0720['age'] < 20), 'ABSI'].describe()

count    11862.000000
mean       157.661478
std         40.817213
min         45.336920
25%        129.454473
50%        163.015035
75%        190.093141
max        266.730539
Name: ABSI, dtype: float64

In [91]:
# 20세 이상 40세 미만 남성

basic0719.loc[(basic0719['sex'] == 1) & (basic0719['age'] >= 20) & (basic0719['age'] < 40), 'ABSI'].describe()

count    9461.000000
mean      201.291365
std        16.065361
min        92.004496
25%       190.041149
50%       200.654995
75%       211.504564
max       271.086585
Name: ABSI, dtype: float64

In [74]:
# 20세 이상 40세 미만 남성

basic0720.loc[(basic0720['sex'] == 1) & (basic0720['age'] >= 20) & (basic0720['age'] < 40), 'ABSI'].describe()

count    10177.000000
mean       201.457266
std         16.043507
min         92.004496
25%        190.202153
50%        200.788905
75%        211.678459
max        271.086585
Name: ABSI, dtype: float64

In [92]:
# 40세 이상 60세 미만 남성

basic0719.loc[(basic0719['sex'] == 1) & (basic0719['age'] >= 40) & (basic0719['age'] < 60), 'ABSI'].describe()

count    12325.000000
mean       199.893153
std         15.914762
min        111.205059
25%        189.114314
50%        199.489850
75%        210.180842
max        298.829851
Name: ABSI, dtype: float64

In [75]:
# 40세 이상 60세 미만 남성

basic0720.loc[(basic0720['sex'] == 1) & (basic0720['age'] >= 40) & (basic0720['age'] < 60), 'ABSI'].describe()

count    13224.000000
mean       200.203990
std         15.988288
min        111.205059
25%        189.374800
50%        199.785725
75%        210.603169
max        298.829851
Name: ABSI, dtype: float64

In [93]:
# 60세 이상 80세 이하 남성

basic0719.loc[(basic0719['sex'] == 1) & (basic0719['age'] >= 60) & (basic0719['age'] <= 80), 'ABSI'].describe()

count    11125.000000
mean       202.083050
std         16.409385
min        134.768120
25%        190.944594
50%        201.570261
75%        212.450844
max        370.244931
Name: ABSI, dtype: float64

In [76]:
# 60세 이상 80세 이하 남성

basic0720.loc[(basic0720['sex'] == 1) & (basic0720['age'] >= 60) & (basic0720['age'] <= 80), 'ABSI'].describe()

count    12106.000000
mean       202.457854
std         16.408176
min        134.768120
25%        191.286295
50%        201.922064
75%        212.828913
max        370.244931
Name: ABSI, dtype: float64

In [94]:
# 20세 미만 여성

basic0719.loc[(basic0719['sex'] == 2) & (basic0719['age'] < 20), 'ABSI'].describe()

count    10425.000000
mean       149.278784
std         35.868245
min         51.829609
25%        125.403522
50%        156.835850
75%        175.969305
max        275.014846
Name: ABSI, dtype: float64

In [77]:
# 20세 미만 여성

basic0720.loc[(basic0720['sex'] == 2) & (basic0720['age'] < 20), 'ABSI'].describe()

count    10854.000000
mean       149.967438
std         35.574097
min         51.829609
25%        127.219080
50%        157.376066
75%        176.229763
max        275.014846
Name: ABSI, dtype: float64

In [95]:
# 20세 이상 40세 미만 여성

basic0719.loc[(basic0719['sex'] == 2) & (basic0719['age'] >= 20) & (basic0719['age'] < 40), 'ABSI'].describe()

count    12445.000000
mean       181.603724
std         16.859798
min         60.517296
25%        170.255757
50%        181.057951
75%        192.387563
max        258.529875
Name: ABSI, dtype: float64

In [78]:
# 20세 이상 40세 미만 여성

basic0720.loc[(basic0720['sex'] == 2) & (basic0720['age'] >= 20) & (basic0720['age'] < 40), 'ABSI'].describe()

count    13225.000000
mean       181.670660
std         16.835336
min         60.517296
25%        170.330030
50%        181.160769
75%        192.402504
max        258.529875
Name: ABSI, dtype: float64

In [96]:
# 40세 이상 60세 미만 여성

basic0719.loc[(basic0719['sex'] == 2) & (basic0719['age'] >= 40) & (basic0719['age'] < 60), 'ABSI'].describe()

count    16369.000000
mean       176.227027
std         15.101636
min         99.736336
25%        165.933712
50%        175.772680
75%        185.925592
max        266.705047
Name: ABSI, dtype: float64

In [79]:
# 40세 이상 60세 미만 여성

basic0720.loc[(basic0720['sex'] == 2) & (basic0720['age'] >= 40) & (basic0720['age'] < 60), 'ABSI'].describe()

count    17494.000000
mean       176.579741
std         15.151306
min         99.736336
25%        166.226984
50%        176.149879
75%        186.357136
max        266.705047
Name: ABSI, dtype: float64

In [97]:
# 60세 이상 80세 이하 여성

basic0719.loc[(basic0719['sex'] == 2) & (basic0719['age'] >= 60) & (basic0719['age'] <= 80), 'ABSI'].describe()

count    14796.000000
mean       174.889770
std         15.488326
min         89.931549
25%        164.397266
50%        174.485710
75%        184.872866
max        255.013609
Name: ABSI, dtype: float64

In [80]:
# 60세 이상 80세 이하 여성

basic0720.loc[(basic0720['sex'] == 2) & (basic0720['age'] >= 60) & (basic0720['age'] <= 80), 'ABSI'].describe()

count    16049.000000
mean       175.341514
std         15.494779
min         89.931549
25%        164.834001
50%        174.973616
75%        185.400655
max        255.013609
Name: ABSI, dtype: float64